In [1]:
import pandas as pd
import altair as alt
import numpy as np
from IPython.display import display

In [2]:
import altair as alt
import numpy as np
import pandas as pd
from sklearn import set_config
from sklearn.compose import make_column_transformer
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.model_selection import (
    GridSearchCV,
    RandomizedSearchCV,
    cross_validate,
    train_test_split,
)
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

# Simplify working with large datasets in Altair
alt.data_transformers.disable_max_rows()

# Output dataframes instead of arrays
set_config(transform_output="pandas")

In [3]:
pip install ucimlrepo

Note: you may need to restart the kernel to use updated packages.


In [4]:
from ucimlrepo import fetch_ucirepo 
  
# fetch dataset 
heart_disease = fetch_ucirepo(id=45) 
  
# data (as pandas dataframes) 
X = heart_disease.data.features 
y = heart_disease.data.targets 

In [5]:
#dataset 
data = pd.read_csv("https://archive.ics.uci.edu/static/public/45/data.csv")
data

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,num
0,63,1,1,145,233,1,2,150,0,2.3,3,0.0,6.0,0
1,67,1,4,160,286,0,2,108,1,1.5,2,3.0,3.0,2
2,67,1,4,120,229,0,2,129,1,2.6,2,2.0,7.0,1
3,37,1,3,130,250,0,0,187,0,3.5,3,0.0,3.0,0
4,41,0,2,130,204,0,2,172,0,1.4,1,0.0,3.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
298,45,1,1,110,264,0,0,132,0,1.2,2,0.0,7.0,1
299,68,1,4,144,193,1,0,141,0,3.4,2,2.0,7.0,2
300,57,1,4,130,131,0,0,115,1,1.2,2,1.0,7.0,3
301,57,0,2,130,236,0,2,174,0,0.0,2,1.0,3.0,1


Define heart disease (positive diagnosis) as num > 0. 

In [33]:
data['heart_disease'] = data['num'] > 0
data

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,num,heart_disease
0,63,1,1,145,233,1,2,150,0,2.3,3,0.0,6.0,0,False
1,67,1,4,160,286,0,2,108,1,1.5,2,3.0,3.0,2,True
2,67,1,4,120,229,0,2,129,1,2.6,2,2.0,7.0,1,True
3,37,1,3,130,250,0,0,187,0,3.5,3,0.0,3.0,0,False
4,41,0,2,130,204,0,2,172,0,1.4,1,0.0,3.0,0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
298,45,1,1,110,264,0,0,132,0,1.2,2,0.0,7.0,1,True
299,68,1,4,144,193,1,0,141,0,3.4,2,2.0,7.0,2,True
300,57,1,4,130,131,0,0,115,1,1.2,2,1.0,7.0,3,True
301,57,0,2,130,236,0,2,174,0,0.0,2,1.0,3.0,1,True


Drop empty rows. 

In [35]:
data1 = data.dropna()
data1

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,num,heart_disease
0,63,1,1,145,233,1,2,150,0,2.3,3,0.0,6.0,0,False
1,67,1,4,160,286,0,2,108,1,1.5,2,3.0,3.0,2,True
2,67,1,4,120,229,0,2,129,1,2.6,2,2.0,7.0,1,True
3,37,1,3,130,250,0,0,187,0,3.5,3,0.0,3.0,0,False
4,41,0,2,130,204,0,2,172,0,1.4,1,0.0,3.0,0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
297,57,0,4,140,241,0,0,123,1,0.2,2,0.0,7.0,1,True
298,45,1,1,110,264,0,0,132,0,1.2,2,0.0,7.0,1,True
299,68,1,4,144,193,1,0,141,0,3.4,2,2.0,7.0,2,True
300,57,1,4,130,131,0,0,115,1,1.2,2,1.0,7.0,3,True


Split data into 75% training, 25% testing. 

In [9]:
from sklearn.model_selection import train_test_split
data_train, data_test = train_test_split(
    data1, train_size = 0.75, random_state = 500
)

Scale numerical variables, passthrough all categorical variables. 

In [10]:
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
data_preprocessor = make_column_transformer(
    (StandardScaler(), ["trestbps", "thalach"]),
    remainder = "passthrough",
    verbose_feature_names_out = True
)
data_preprocessor

ColumnTransformer(remainder='passthrough',
                  transformers=[('standardscaler', StandardScaler(),
                                 ['trestbps', 'thalach'])])

Run preliminary k = 3, generate a confusion matrix. 

In [11]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import make_pipeline

knn = KNeighborsClassifier(n_neighbors = 3)

X_train = data_train[["sex", "fbs", "trestbps", "restecg", "cp", "thal", "thalach"]]
y_train = data_train["heart_disease"]

knn_pipeline = make_pipeline(data_preprocessor, knn)
knn_pipeline.fit(X_train, y_train)

data_test_predictions = data_test.assign(
    predict = knn_pipeline.predict(data_test[["sex", "fbs", "trestbps", "restecg", "cp", "thal", "thalach"]])
)
data_test_predictions

X_test = data_test[["sex", "fbs", "trestbps", "restecg", "cp", "thal", "thalach"]]
y_test = data_test["heart_disease"]

data_prediction_accuracy = knn_pipeline.score(X_test, y_test)
data_prediction_accuracy

data_mat = pd.crosstab(
    data_test_predictions["heart_disease"], 
    data_test_predictions["predict"]
)
data_mat

predict,False,True
heart_disease,,
False,33,5
True,10,27


Set random seed, cross-validate the model and try to find the k-value with greatest recall. 

In [37]:
np.random.seed(1234)

data_pipe = make_pipeline(data_preprocessor, knn)
data_vfold_score = pd.DataFrame(
    cross_validate(
        estimator = data_pipe, 
        cv = 10, 
        X = X_train, 
        y = y_train, 
        return_train_score = True
    )
)
data_vfold_score

data_metrics = data_vfold_score.agg(["mean", "sem"])
data_metrics

param_grid = {
    "kneighborsclassifier__n_neighbors": range(1, 50, 1)
}

data_tune_pipe = make_pipeline(data_preprocessor, KNeighborsClassifier())

data_tune_grid_recall = GridSearchCV(
    estimator = data_tune_pipe,
    param_grid = param_grid, 
    cv = 10, 
    return_train_score = True,
    scoring = "recall",
    n_jobs = -1,
)

model_grid_recall = data_tune_grid_recall.fit(X_train, y_train)
accuracies_grid_recall = pd.DataFrame(model_grid_recall.cv_results_)
accuracies_grid_recall

cv_plot_recall = alt.Chart(accuracies_grid_recall).mark_line(point=True).encode(
    x=alt.X("param_kneighborsclassifier__n_neighbors").title("K"),
    y=alt.Y("mean_test_score").title("Recall Estimate").scale(zero=False)
)
cv_plot_recall

alt.Chart(...)

Run again on the test data with k = 17. 

In [40]:
knn1 = KNeighborsClassifier(n_neighbors = 17)

knn_pipeline1 = make_pipeline(data_preprocessor, knn1)
knn_pipeline1.fit(X_train, y_train)

data_test_predictions1 = data_test.assign(
    predict = knn_pipeline1.predict(data_test[["sex", "fbs", "trestbps", "restecg", "cp", "thal", "thalach"]])
)
data_test_predictions1

data_test_predictions1["predicted"] = data_test_predictions1["predict"] > 0
data_test_predictions1

data_prediction_accuracy = knn_pipeline1.score(X_test, y_test)
data_prediction_accuracy

data_mat1 = pd.crosstab(
    data_test_predictions1["heart_disease"], 
    data_test_predictions1["predicted"]
)
data_mat1

predicted,False,True
heart_disease,,
False,29,9
True,7,30


Recall of k = 17. 

In [41]:
from sklearn.metrics import recall_score, precision_score
recall_score(
    y_true = data_test_predictions1["heart_disease"], 
    y_pred = data_test_predictions1["predicted"], 
    pos_label = True
)

0.8108108108108109

Accuracy of k = 17. 

In [42]:
data_prediction_accuracy = knn_pipeline1.score(X_test, y_test)
data_prediction_accuracy

0.7866666666666666

In [16]:
from sklearn.metrics import recall_score, precision_score

data_test["predicted"] = data_tune_grid.predict(
    data_test[["sex", "fbs", "trestbps", "restecg", "cp", "thal", "thalach"]]
)

recall_score(
    y_true = data_test["heart_disease"], 
    y_pred = data_test["predicted"], 
    pos_label = True
)

0.8108108108108109

Accuracy estimate versus k-value. 

In [43]:
np.random.seed(1234)

data_tune_grid_acc = GridSearchCV(
    estimator = data_tune_pipe,
    param_grid = param_grid, 
    cv = 10, 
    return_train_score = True,
    n_jobs = -1,
)
data_tune_grid_acc

model_grid_acc = data_tune_grid_acc.fit(X_train, y_train)
accuracies_grid_acc = pd.DataFrame(model_grid_acc.cv_results_)
accuracies_grid_acc

cv_plot_acc = alt.Chart(accuracies_grid_acc).mark_line(point=True).encode(
    x=alt.X("param_kneighborsclassifier__n_neighbors").title("K"),
    y=alt.Y("mean_test_score").title("Accuracy Estimate").scale(zero=False)
)
cv_plot_acc

alt.Chart(...)

In [17]:
data_test_predictions1

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,num,heart_disease,predict,predicted
139,51,1,3,125,245,1,2,166,0,2.4,2,0.0,3.0,0,False,False,False
205,45,1,4,142,309,0,2,147,1,0.0,2,3.0,7.0,3,True,True,True
169,45,0,2,112,160,0,0,138,0,0.0,2,0.0,3.0,0,False,False,False
196,69,1,1,160,234,1,2,131,0,0.1,2,1.0,3.0,0,False,False,False
119,65,1,4,135,254,0,2,127,0,2.8,2,1.0,7.0,2,True,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
132,29,1,2,130,204,0,2,202,0,0.0,1,0.0,3.0,0,False,False,False
74,44,1,4,110,197,0,2,177,0,0.0,1,1.0,3.0,1,True,False,False
290,67,1,3,152,212,0,2,150,0,0.8,2,0.0,7.0,1,True,True,True
134,43,0,3,122,213,0,0,165,0,0.2,2,0.0,3.0,0,False,False,False


In [18]:
data_train

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,num,heart_disease
283,35,1,2,122,192,0,0,174,0,0.0,1,0.0,3.0,0,False
155,70,1,4,130,322,0,2,109,0,2.4,2,3.0,3.0,1,True
125,45,0,2,130,234,0,2,175,0,0.6,2,0.0,3.0,0,False
25,50,0,3,120,219,0,0,158,0,1.6,2,0.0,3.0,0,False
254,43,1,4,115,303,0,0,181,0,1.2,2,0.0,3.0,0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
240,41,1,2,110,235,0,0,153,0,0.0,1,0.0,3.0,0,False
72,62,1,4,120,267,0,0,99,1,1.8,2,2.0,7.0,1,True
292,44,1,4,120,169,0,0,144,1,2.8,3,0.0,6.0,2,True
209,62,0,4,150,244,0,0,154,1,1.4,2,0.0,3.0,1,True


In [19]:
knn2 = KNeighborsClassifier(n_neighbors = 1)

knn_pipeline2 = make_pipeline(data_preprocessor, knn1)
knn_pipeline2.fit(X_train, y_train)

data_test_predictions2 = data_test.assign(
    predict = knn_pipeline2.predict(data_test[["sex", "fbs", "trestbps", "restecg", "cp", "thal", "thalach"]])
)
data_test_predictions2

data_test_predictions2["predicted"] = data_test_predictions2["predict"] > 0
data_test_predictions2

data_prediction_accuracy = knn_pipeline2.score(X_test, y_test)
data_prediction_accuracy

data_mat2 = pd.crosstab(
    data_test_predictions2["heart_disease"], 
    data_test_predictions2["predicted"]
)
data_mat2

predicted,False,True
heart_disease,,
False,29,9
True,7,30


In [20]:
data_test_predictions2

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,num,heart_disease,predicted,predict
139,51,1,3,125,245,1,2,166,0,2.4,2,0.0,3.0,0,False,False,False
205,45,1,4,142,309,0,2,147,1,0.0,2,3.0,7.0,3,True,True,True
169,45,0,2,112,160,0,0,138,0,0.0,2,0.0,3.0,0,False,False,False
196,69,1,1,160,234,1,2,131,0,0.1,2,1.0,3.0,0,False,False,False
119,65,1,4,135,254,0,2,127,0,2.8,2,1.0,7.0,2,True,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
132,29,1,2,130,204,0,2,202,0,0.0,1,0.0,3.0,0,False,False,False
74,44,1,4,110,197,0,2,177,0,0.0,1,1.0,3.0,1,True,False,False
290,67,1,3,152,212,0,2,150,0,0.8,2,0.0,7.0,1,True,True,True
134,43,0,3,122,213,0,0,165,0,0.2,2,0.0,3.0,0,False,False,False


In [21]:
np.random.seed(2000)

data_pipe = make_pipeline(data_preprocessor, knn)
data_vfold_score = pd.DataFrame(
    cross_validate(
        estimator = data_pipe, 
        cv = 5, 
        X = X_train, 
        y = y_train, 
        return_train_score = True
    )
)
data_vfold_score

data_metrics = data_vfold_score.agg(["mean", "sem"])
data_metrics

param_grid = {
    "kneighborsclassifier__n_neighbors": range(1, 50, 1)
}

data_tune_pipe = make_pipeline(data_preprocessor, KNeighborsClassifier())

data_tune_grid = GridSearchCV(
    estimator = data_tune_pipe,
    param_grid = param_grid, 
    cv = 5, 
    return_train_score = True, 
    n_jobs = -1,
)
data_tune_grid

model_grid = data_tune_grid.fit(X_train, y_train)
accuracies_grid = pd.DataFrame(model_grid.cv_results_)
accuracies_grid

cv_plot = alt.Chart(accuracies_grid).mark_line(point=True).encode(
    x=alt.X("param_kneighborsclassifier__n_neighbors").title("K"),
    y=alt.Y("mean_test_score").title("Accuracy Estimate").scale(zero=False)
)
cv_plot

alt.Chart(...)

In [22]:
knn2 = KNeighborsClassifier(n_neighbors = 2)

knn_pipeline2 = make_pipeline(data_preprocessor, knn2)
knn_pipeline2.fit(X, y)

data_test_predictions2 = data_test.assign(
    predict = knn_pipeline2.predict(data_test[["sex", "fbs", "trestbps", "restecg", "cp", "thal", "thalach"]])
)
data_test_predictions2

data_test_predictions2["predicted"] = data_test_predictions2["predict"] > 0
data_test_predictions2

data_prediction_accuracy = knn_pipeline2.score(X_test, y_test)
data_prediction_accuracy

data_mat2 = pd.crosstab(
    data_test_predictions2["heart_disease"], 
    data_test_predictions2["predicted"]
)
data_mat2

ValueError: Input X contains NaN.
KNeighborsClassifier does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

In [ ]:
X_test = data_test[["sex", "fbs", "trestbps", "restecg", "cp", "thal", "thalach"]]
y_test = data_test["heart_disease"]
data_prediction_accuracy = knn_pipeline2.score(X_test, y_test)
data_prediction_accuracy

In [ ]:
knn_spec = KNeighborsClassifier(n_neighbors = 3)

X = fruit_train[["mass", "color_score"]]
y = fruit_train["fruit_name"]

fruit_fit = make_pipeline(fruit_preprocessor, knn_spec).fit(X, y)
fruit_fit

fruit_test_predictions = fruit_test.assign(
    predicted=fruit_fit.predict(fruit_test[["mass", "color_score"]])
)
fruit_test_predictions

In [ ]:
data_preprocessed = data_preprocessor.fit_transform(data1)
data_preprocessed